In [1]:
import numpy as np
import pandas as pd
import time
import cv2
import threading

In [2]:
def hod_rd_sub1(mean):
    for i in range(my,shape[1]-lmd,lmd-2):
        ver_rd(i,mean)

In [3]:
def hod_rd_sub2(mean1):
    for i in range(my+lmd,0, -(lmd-2)):
        ver_rd(i,mean1)

In [70]:
def hor_rd(mean):
    hod_rd_sub1(mean)
    hod_rd_sub2(mean)

In [78]:
def ver_rd(x,mean):
    lm1 = lim1
    lm2 = lim2
    for i in range(mx,lmd+1,-(lmd-10)):
        lm1 += lm1*0.03
        lm2 += lm2*0.03
        low = mean - [0,lm1,lm2]
        high = mean + [180,lm1,lm2]
        newscr[i-lmd:i, x:x+lmd] = cv2.inRange(hls[i-lmd:i, x:x+lmd],low,high)
        res = [0]+[np.mean(hls[i-lmd:i, x:x+lmd,j]) for j in range(1,3)]
        res = np.array(res)
        if(abs(res[1]-mean[1])<=15 and abs(res[2]-mean[2])<=15):
            mean = (4*res+6*mean)/10

In [81]:
import numpy as np
import pandas as pd
import time
import cv2
import threading
# lim = 4*std
lmd = 40

kernel = np.ones((3,3),np.uint8)
vrec = cv2.VideoCapture('v4.mp4',0)
shape = [480,680]#scr.shape

mx = shape[0]
my = shape[1]//2
print("running")
kkk = 300*8#*6
mask_prev1 = np.zeros((shape[0],shape[1]),np.uint8)
mask_prev2 = np.zeros((shape[0],shape[1]),np.uint8)
while(kkk):
    kkk-=1
    ret, scr = vrec.read()

while True:
    start_time = time.time()
    ret, scr = vrec.read()
    ret, scr = vrec.read()
#     ret, scr2 = vrec.read()
#     ret, scr3 = vrec.read()
    
#     scr = np.mean(scr1,scr2,scr3)
    if not ret:
            break
    scr = cv2.resize(scr, (680,480)) 
    
    newscr = np.zeros((shape[0],shape[1]),np.uint8)
    scr = cv2.bilateralFilter(scr,7,12,12)
    hls = cv2.cvtColor(scr, cv2.COLOR_BGR2HLS)
    mean = [0]+[np.mean(hls[440:480,200:480, i]) for i in range(1,3)]
    std = [np.std(hls[440:480,200:480, i]) for i in range(1,3)]
    lim1 = (lim1 + 16*std[0])/5#(1:4(4))/5
    lim2 = (lim2 + 16*std[1])/5
    print(lim1,lim2)
    mean = np.array(mean)
    print(mean)
    hor_rd(mean)
    band =  cv2.bitwise_and(scr,scr,mask = newscr)
    
    gray = cv2.cvtColor(scr, cv2.COLOR_BGR2GRAY)
    mean1 = np.mean(gray[440:480,200:480])
    mean1 = np.array(mean1)
    low1 = mean1 - 25
    high1 = mean1 + 25
    
    mask1 = cv2.inRange(gray,low1,high1)
    band1 =  cv2.bitwise_and(scr,scr,mask = mask1)
    
#     newscr = cv2.morphologyEx(newscr,cv2.MORPH_OPEN,kernel, iterations = 1)
#     newscr = cv2.morphologyEx(newscr,cv2.MORPH_CLOSE,kernel, iterations = 1)

    
        
    mask3 = newscr+mask1
#     band3 =  cv2.bitwise_and(scr,scr,mask = mask3)
    
    mask4 = mask_prev1 + mask3 #+ mask_prev2
#     mask_pres2 = mask_prev1
    mask_prev1 = mask3 
    
#     canny = cv2.Canny(gray,10,50)
#     cv2.imshow("canny", canny)

#     mask4 = cv2.morphologyEx(mask4,cv2.MORPH_OPEN,kernel, iterations = 1)
#     mask4 = cv2.morphologyEx(mask4,cv2.MORPH_CLOSE,kernel, iterations = 1)
    
    
#     cv2.imshow('blur',band)
#     cv2.imshow('mean1',band1)
#     cv2.imshow('band3',band3)
    cv2.imshow('newscr',newscr)
    cv2.imshow('mean1',mask1)
    cv2.imshow('band3',mask4)
#     mask4 = cv2.morphologyEx(mask4,cv2.MORPH_OPEN,kernel, iterations = 1)
#     mask4 = cv2.morphologyEx(mask4,cv2.MORPH_CLOSE,kernel, iterations = 1)


    dist_transform = cv2.distanceTransform(mask4,cv2.DIST_L2,5)
    ret, mask4 = cv2.threshold(dist_transform,0.07*dist_transform.max(),255,0)
    mask4 = cv2.morphologyEx(mask4,cv2.MORPH_CLOSE,kernel, iterations = 1)
    ret, mask4 = cv2.threshold(dist_transform,0.07*dist_transform.max(),255,0)
    cv2.imshow('band4',mask4)
    mask4 = np.uint8(mask4)
    im2, contours, hierarchy = cv2.findContours(mask4, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     cnt = contours[0]
    scr1 = cv2.drawContours(scr, contours, -1, (0,255,0), 3)
    
    
#     n_mask4 = cv2.bitwise_not(mask4)
    
#     im2, contours, hierarchy = cv2.findContours(n_mask4, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# #     cnt = contours[0]
#     scr2 = cv2.drawContours(scr, contours, -1, (0,255,0), 3)
    
    
    cv2.imshow('scr1',scr1)
#     cv2.imshow('scr2',scr2)
    print("FPS: ", 1.0 / (time.time() - start_time))
#     time.sleep(0.2)
    q=cv2.waitKey(1)
    if q == 27:
        cv2.destroyAllWindows()
        break
cv2.destroyAllWindows()

running
46.7899479151 26.9977599651
[   0.          150.31910714   27.19401786]
FPS:  2.500206547157428
47.3560632786 27.1375140307
[   0.          149.49         26.79223214]
FPS:  22.295540683488994
43.4601338651 28.2622552395
[   0.          149.76098214   27.19964286]
FPS:  23.167574375006904
43.5792839052 29.280423317
[   0.          151.00446429   27.60776786]
FPS:  23.877127665630585
45.6969720455 29.0297361879
[   0.          150.42303571   27.61276786]
FPS:  22.548445538752667
41.101314425 28.501259349
[   0.          149.85607143   27.75535714]
FPS:  23.743852180607764
51.0661425631 38.4279492217
[   0.          151.30107143   30.20419643]
FPS:  24.511036179500817
34.3116698815 58.2674050521
[   0.          154.97839286   34.96375   ]
FPS:  23.81476476533312
25.2331999952 68.376126721
[   0.          155.214375     37.33723214]
FPS:  21.892310583126292
23.8958521755 73.6989294373
[   0.          155.37517857   38.33303571]
FPS:  22.497647948592792
21.1657126087 74.2019741645


21.4194078523 18.9327344861
[  0.          71.75866071  56.82223214]
FPS:  23.911430363149194
63.5190496953 39.9218410399
[  0.          80.71883929  53.40571429]
FPS:  24.051839023774843
26.3146362541 34.6515073167
[  0.          70.48160714  59.06857143]
FPS:  23.451649156551056
11.532386486 26.9117152251
[  0.          71.64544643  57.34508929]
FPS:  24.805890528432446
8.38918346188 23.5184467059
[  0.       71.85     57.30875]
FPS:  23.808276096951808
7.79281553159 20.6233881487
[  0.          71.96053571  56.74339286]
FPS:  24.261781497827936
9.55822105309 23.4491608126
[  0.          72.23848214  57.03375   ]
FPS:  23.52385866517106
58.0039607938 40.6309964095
[  0.          78.19705357  54.913125  ]
FPS:  24.24369097025537
18.1196927547 25.5859356897
[  0.          72.47339286  57.8675    ]
FPS:  21.86309709972686
11.3602278001 27.8182089328
[  0.          72.50883929  58.65142857]
FPS:  21.991139167501192
11.3576642073 20.8795520389
[  0.          72.92116071  58.033125  ]
FPS:

25.9716130393 29.6702206786
[  0.          62.89544643  59.42875   ]
FPS:  22.335790056660844
26.7649848005 26.0694891022
[  0.       64.0475   58.34875]
FPS:  22.49209830597547
26.7349748301 26.4860577311
[  0.          63.20821429  59.69339286]
FPS:  23.55318455957502
26.2253789816 24.9715112827
[  0.          64.88205357  58.42598214]
FPS:  23.88569410987534
48.5552447337 36.3141430099
[  0.          70.97732143  54.63267857]
FPS:  23.811114454240442
28.0242162293 24.0073786651
[  0.          65.94508929  58.45330357]
FPS:  24.520780352058743
22.2755870585 18.6180903247
[  0.          65.32785714  58.98258929]
FPS:  24.069091764651873
20.9104938149 17.4547944939
[  0.          65.75125     58.44598214]
FPS:  23.302983499083282
18.8306154783 14.8443976155
[  0.          65.73607143  58.00026786]
FPS:  23.734312665870675
17.2763305745 11.1267103222
[  0.          67.52410714  56.14258929]
FPS:  24.22408834163076
18.9916815841 11.8026045552
[  0.          67.12866071  57.29669643]
FPS:

41.2483430619 19.7746781292
[  0.          95.70526786  42.61205357]
FPS:  22.283695403856065
31.3633839576 13.3988652775
[  0.          87.34053571  46.84651786]
FPS:  23.663076654010414
27.2402307427 11.2164010568
[  0.          87.39410714  47.50196429]
FPS:  20.715374395598424
23.5603128026 11.0709320867
[  0.          83.54883929  47.61758929]
FPS:  23.583777066804615
24.056571762 16.2779462972
[  0.          81.93607143  49.81080357]
FPS:  20.377911439759796
14.0719961054 16.773025268
[  0.          77.31830357  53.49142857]
FPS:  21.211635716308614
11.6424249786 13.4718970644
[  0.          73.20758929  56.14491071]
FPS:  22.98740004713336
8.20663200349 14.5969932778
[  0.          72.29732143  56.98107143]
FPS:  21.270800158226244
12.1637138228 20.8491114388
[  0.          71.59696429  57.85821429]
FPS:  23.043600562587905
16.1481092815 21.2974735224
[  0.          68.91910714  59.72705357]
FPS:  23.269370319001386
10.8327315907 21.0826045531
[  0.          70.00508929  59.1908

8.72848027771 9.98333688416
[  0.          91.44357143  49.105625  ]
FPS:  21.18410246826908
8.63683392551 12.7856624313
[  0.        95.601875  46.271875]
FPS:  23.548291833927518
40.1065569277 25.2061473479
[   0.          114.294375     33.34633929]
FPS:  20.92527514193632
14.8304481045 15.8040611697
[  0.          99.775       40.38366071]
FPS:  23.361780580047565
14.8606519182 14.3562669931
[   0.          104.32232143   37.62223214]
FPS:  19.198271639973818
21.3322916563 16.2516260819
[   0.          113.06392857   36.225625  ]
FPS:  22.292578183132427
19.0453736929 13.6190290852
[   0.          112.24660714   36.27821429]
FPS:  22.2376188279705
19.1936091807 11.2259309583
[   0.          110.109375     38.18053571]
FPS:  23.81476476533312
13.4672399538 9.7871865272
[   0.          109.9175       39.40383929]
FPS:  20.76162002158181
11.2854339863 9.1872197901
[   0.          108.09303571   41.06678571]
FPS:  24.88005172587658
10.0844802227 10.79002135
[   0.          106.22160714

23.8602817971 21.7492934194
[   0.          109.41026786   41.850625  ]
FPS:  23.197044443952834
18.6481867969 18.7606701553
[   0.          106.44196429   41.99473214]
FPS:  23.02551068022991
18.5653116038 17.6409981976
[   0.          102.444375     42.99848214]
FPS:  23.493947694186314
14.0155923211 15.3532856547
[  0.          94.91285714  45.00160714]
FPS:  24.21010586102998
18.1745652937 18.1448616986
[  0.          88.76946429  47.86491071]
FPS:  21.909120825737432
15.3118426943 18.4616227861
[  0.          85.20625     48.94348214]
FPS:  20.754223282235
13.6419659657 17.0616312387
[  0.        80.718125  52.07375 ]
FPS:  21.240423764863166
11.1859548382 17.445442253
[  0.          74.66026786  55.94857143]
FPS:  22.992188485004633
12.3159968412 19.034152149
[  0.          70.07339286  59.66267857]
FPS:  20.450741858571476
7.84410480756 22.7621828245
[  0.          78.97428571  48.13651786]
FPS:  21.389470248658792
8.97407222019 26.0153370194
[  0.          88.70116071  40.71116

In [78]:
import numpy as np
import pandas as pd
import time
import cv2
import threading

vrec = cv2.VideoCapture('v2.mp4',0)
shape = [480,680]

mx = shape[0]
my = shape[1]//2
print("running")
kkk = 300*6*6
while(kkk):
    kkk-=1
    ret, scr = vrec.read()

ret, scr1 = vrec.read()
scr1 = cv2.resize(scr1, (680,480)) 
scr1 = cv2.cvtColor(scr1, cv2.COLOR_BGR2GRAY)
ret, scr2 = vrec.read()
scr2 = cv2.resize(scr2, (680,480)) 
scr2 = cv2.cvtColor(scr2, cv2.COLOR_BGR2GRAY)
while True:
    start_time = time.time()
    scr = scr1
    scr1 = scr2
    ret, scr2 = vrec.read()
#     ret, scr3 = vrec.read()
#     scr = np.mean(scr1,scr2,scr3)
    if not ret:
            break
    scr2 = cv2.resize(scr2, (680,480)) 
    scr2 = cv2.cvtColor(scr2, cv2.COLOR_BGR2GRAY)
    
    d0 = abs(scr1-scr)
    d1 = abs(scr2-scr)
    d2=d0+d1
    
    cv2.imshow('d0',d0)
    cv2.imshow('d1',d1)
    cv2.imshow('d2',d2)
#     cv2.imshow('d3',d3)
#     cv2.imshow('d4',d4)

    cv2.imshow('scr1',scr)
    print("FPS: ", 1.0 / (time.time() - start_time))
    q=cv2.waitKey(1)
    if q == 27:
        cv2.destroyAllWindows()
        break
cv2.destroyAllWindows()

running
FPS:  6.805267291005497
FPS:  237.974695035461
FPS:  213.0818939239992
FPS:  233.77014825548991
FPS:  230.83676389653274
FPS:  211.23609991941981
FPS:  220.8458298230834
FPS:  214.39983642590605
FPS:  202.9174649250121
FPS:  214.3669630992538
FPS:  199.08410860072146
FPS:  201.45552353506244
FPS:  143.248087431694
FPS:  175.0471182337966
FPS:  165.47536197577622
FPS:  182.2421898761677
FPS:  90.95708368572853
FPS:  185.3590242177833
FPS:  185.9754356404913
FPS:  168.23649271990695
FPS:  185.3180753766624
FPS:  205.09041122683487
FPS:  223.36265843007774
FPS:  225.17335051269663
FPS:  155.42518342844437
FPS:  178.0945182794786
FPS:  232.61627197604128
FPS:  183.5260348297891
FPS:  243.2608746085141
FPS:  241.13510405887087
FPS:  225.84018953262978
FPS:  171.67958740943882
FPS:  217.8180307436643
FPS:  209.98818464003205
FPS:  222.87602954460917
FPS:  169.02974127508665
FPS:  147.40648063541155
FPS:  184.47853624208304
FPS:  212.76842692639374
FPS:  215.47927048548678
FPS:  157.6